# This is where we will specifically put all code and answers for our third analysis question

## Question: How do long-term trends in unemployment rates influence the overall performance and risk profile of the S&P 500 index.  

Below is all imports needed to run our code.

In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

Below is the web-scraping process.

In [ ]:
browser = webdriver.Chrome()
browser.get("https://www.bls.gov/charts/employment-situation/civilian-unemployment-rate.htm")
browser.maximize_window()
wait = WebDriverWait(browser, 15)
show_table_button = wait.until(
    EC.element_to_be_clickable((By.LINK_TEXT, "Show table"))
)

show_table_button.click()

time.sleep(2)

table = browser.find_element(By.TAG_NAME, "table")
rows = table.find_elements(By.TAG_NAME, "tr")

month_years, total_rates, men_rates, women_rates, teen_rates, white_rates, black_rates, asian_rates, latino_rates = ([] for _ in range(9))

for row in rows[1:]:
    cols = row.find_elements(By.TAG_NAME, "td")
    if len(cols) == 8:
        th = row.find_element(By.TAG_NAME, "th")
        month_year = th.find_element(By.CLASS_NAME, "sub0").text
        month_years.append(month_year)
        
        total_rates.append(cols[0].text)
        men_rates.append(cols[1].text)
        women_rates.append(cols[2].text)
        teen_rates.append(cols[3].text)
        white_rates.append(cols[4].text)
        black_rates.append(cols[5].text)
        asian_rates.append(cols[6].text)
        latino_rates.append(cols[7].text)
        unemployment_df = pd.DataFrame({
                "Date": month_years,
    "Total Rate": total_rates,
    "Male Rate": men_rates,
    "Female Rate": women_rates,
    "Teen Rate": teen_rates,
    "White Rate": white_rates,
    "Black Rate": black_rates,
    "Asian Rate": asian_rates,
    "Hispanic Rate": latino_rates
})

browser.quit()

print(unemployment_df.head(10))

In [ ]:
sp500_df = pd.read_csv("spy.csv")
sp500_df.head(10)

Cleaning the data types.

In [ ]:
unemployment_df["Date"] = pd.to_datetime(unemployment_df["Date"]).dt.strftime('%b %Y')
unemployment_df["Total Rate"] = pd.to_numeric(unemployment_df["Total Rate"], errors='coerce')
unemployment_df["Male Rate"] = pd.to_numeric(unemployment_df["Male Rate"], errors='coerce')
unemployment_df["Female Rate"] = pd.to_numeric(unemployment_df["Female Rate"], errors='coerce')
unemployment_df["Teen Rate"] = pd.to_numeric(unemployment_df["Teen Rate"], errors='coerce')
unemployment_df["White Rate"] = pd.to_numeric(unemployment_df["White Rate"], errors='coerce')
unemployment_df["Black Rate"] = pd.to_numeric(unemployment_df["Black Rate"], errors='coerce')
unemployment_df["Asian Rate"] = pd.to_numeric(unemployment_df["Asian Rate"], errors='coerce')
unemployment_df["Hispanic Rate"] = pd.to_numeric(unemployment_df["Hispanic Rate"], errors='coerce')

unemployment_df.dtypes

unemployment_df['Unemployment Change'] = unemployment_df['Total Rate'].pct_change().round(4)
unemployment_df.loc[0, 'Unemployment Change'] = 0

unemployment_df.head(10)

In [ ]:
sp500_df.dtypes
sp500_df["Date"] = pd.to_datetime(sp500_df["Date"])
sp500_df.dtypes

sp500_df = sp500_df[(sp500_df["Day"] == 1) & (sp500_df["Date"] >= "2005-03-01")]
sp500_df = sp500_df.reset_index(drop = True)

sp500_df["Date"] = sp500_df["Date"].dt.strftime('%b %Y')
sp500_df = sp500_df.drop(columns=['Day', 'Weekday', 'Week', 'Month', 'Year'])

sp500_df["Close Change"] = sp500_df["Close"].pct_change().round(4)
sp500_df.loc[0, "Close Change"] = 0
sp500_df.head(10)

Merging the DataFrames

In [ ]:
sp500_df['Date'] = pd.to_datetime(sp500_df['Date'], format='%b %Y')
unemployment_df['Date'] = pd.to_datetime(unemployment_df['Date'], format='%b %Y')

merged_df = pd.merge(sp500_df, unemployment_df, on='Date', how='inner')

merged_df = merged_df.sort_values('Date').reset_index(drop=True)

print(merged_df.head())

This code filters the data to focus on the period between January 1, 2007, and December 31, 2010, representing the Great Recession. It then plots the unemployment rates for various demographic groups (e.g., male, female, white, black, Asian, Hispanic) over time during this period. Each demographic group's unemployment rate is visualized as a line plot with markers, and the chart includes labels, a title, and a legend to distinguish the different demographic groups.

In [ ]:
recession_df = merged_df[(merged_df['Date'] >= '2007-01-01') & (merged_df['Date'] <= '2010-12-31')]

demographic_columns = ['Male Rate', 'Female Rate', 'White Rate', 'Black Rate', 'Asian Rate', 'Hispanic Rate']

plt.figure(figsize=(14, 7))

for column in demographic_columns:
    plt.plot(recession_df['Date'], recession_df[column], marker='o', label=column)

plt.title('Unemployment Rates by Demographic During Recession (2007–2010)')
plt.xlabel('Date')
plt.ylabel('Unemployment Rate (%)')
plt.grid(True)
plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()

This code performs independent t-tests to compare the unemployment rates for different demographic groups during the period of the Great Recession (2008-2009) against the period just before the recession (2006-2007). The results for each demographic group (e.g., male, female, white, black, Asian, Hispanic) include the t-statistic, p-value, and whether the difference in means is statistically significant (p-value < 0.05). The results are stored in a dictionary and then displayed as a DataFrame, where each demographic's results are shown in rows.

In [ ]:
pre_recession = merged_df[(merged_df['Date'] >= '2006-01-01') & (merged_df['Date'] <= '2007-12-31')]
during_recession = merged_df[(merged_df['Date'] >= '2008-01-01') & (merged_df['Date'] <= '2009-12-31')]

demographics = ['Male Rate', 'Female Rate', 'White Rate', 'Black Rate', 'Asian Rate', 'Hispanic Rate']

t_test_results = {}

for demo in demographics:
    stat, p_value = ttest_ind(during_recession[demo], pre_recession[demo], equal_var=False)
    t_test_results[demo] = {
        't_statistic': stat,
        'p_value': p_value,
        'significant': p_value < 0.05
    }
pd.DataFrame(t_test_results).T

This code generates a scatter plot that visualizes the relationship between the S&P 500 closing prices (Close) and the total unemployment rate (Total Rate). It also calculates and prints the correlation coefficient between these two variables, indicating the strength and direction of their linear relationship. If the correlation is positive, it suggests that as the S&P 500 rises, the unemployment rate tends to increase (or vice versa). If negative, it indicates an inverse relationship between the two.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x=merged_df['Close'], y=merged_df['Total Rate'])
plt.title('S&P 500 vs Unemployment Rate')
plt.xlabel('S&P 500')
plt.ylabel('Unemployment Rate')
plt.show()

correlation = merged_df['Close'].corr(merged_df['Total Rate'])
print(f'Correlation between S&P 500 and Unemployment Rate: {correlation}')